<a href="https://colab.research.google.com/github/thiagonobrega/tl-pprl/blob/master/notebooks/tl%40pprl2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
import pandas as pd
import time
import logging

from multiprocessing import Pool

from tqdm.notebook import trange, tqdm

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import recall_score, precision_score , f1_score , accuracy_score

In [ ]:
# import multiprocessing
# multiprocessing.set_start_method('forkserver')
from google.colab import auth
import gspread
from google.auth import default

import warnings
warnings.simplefilter(action='ignore')

# configure env

In [ ]:
#cuidado com a branch
!rm -rf tl-pprl/ tlpprl/

!git clone https://github.com/thiagonobrega/tl-pprl.git &> /dev/null
sys.path.append("./tl-pprl/")

In [ ]:
# !cp -r tl-pprl/ tlpprl
# sys.path.append("./tlpprl/")

In [ ]:
# nao funciona
# !pip uninstall --yes gdown  &> /dev/null
# !pip install gdown==4.4.0 --no-cache-dir &> /dev/null
# !pip install --upgrade gdown &> /dev/null
#alternativa
!pip install --user gdown==4.4.0  &> /dev/null

In [ ]:
!rm -rf datasets
!mkdir datasets

personal_ds_links = [
                     '1J-4xHTanKik7Nytm5T_Y7yYiBqi3Ej9a', #yv-er
                     '1ugVFwn93bAQrxp5p7hICJRAv2igqUXNH', #tse
                     '1TWT-Veh786NVal7Rz8VK1xzaJWmv6Nsd', #ncvr
                     '1V9SLWLFjsySHYTjpzMcMlM23atQTPc9D', #mvr
                     '1iKlLBiAJq7UvT3woOcV-zrQHKN6QKubA', #census
]

for id in tqdm(personal_ds_links,leave=False):
  !gdown {id} --folder -c --output datasets/ --no-cookies &> /dev/null


  0%|          | 0/5 [00:00<?, ?it/s]

```
/content# du -h --max-depth=1 datasets/
759M    datasets/tse
688K    datasets/yv-er
168M    datasets/ncvr
139M    datasets/mvr
3.7M    datasets/census
1.1G    datasets/
```

In [ ]:
#@title Nao utilizado
# !rm -rf datasets
# !mkdir datasets

# #faltando os restaurantes
# ids1 = [ "1T_prZFZc4__vZbLK2aDqkQbGNRIU-Ma-","1DYGHcvCwKBuQTVdLAoxqOKthFPvVjSRq",
#          "1Xh4RsZoVDz04ohszr5J2QWgFEuauYJw3",
#          "1VAGYqb05C9kj-8FgSu8P6f58hgLFdh7n","1TsrO9dIrHBUyC8jxf6Nb_sfY9YMsl4Yl",
#          "1yB1_kzYP0HkZzSan7EtzM2MlAvnE_f_t","1r86bL6JV6DyINxZOxIB-rjPJ9Vy0imei" #dblp-acm
# ]
# for id in tqdm(ids1,leave=False):
#   # !gdown --id {id} --folder -c --output datasets/ --no-cookies &> /dev/null
#   url = 'https://drive.google.com/open?id='
#   id = url+id
#   # !/root/.local/bin/gdown {id} --folder -c --output datasets/ --no-cookies &> /dev/null
#   !gdown {id} --folder -c --output datasets/ --no-cookies &> /dev/null
  

# #census, books ,abt-buy , restaurants
# ids2 = ["https://drive.google.com/drive/folders/1BImatx4-7WrE_eVZ7zPUq408fiUEfq8w",
#         "https://drive.google.com/drive/folders/14ow2HnOtcxF6EcXuxmtlXyMaxkB3Uew1"
#         "https://drive.google.com/drive/folders/1BImatx4-7WrE_eVZ7zPUq408fiUEfq8w",
#         "https://drive.google.com/drive/folders/1wEA-fSQsxLWvu3HlHzPH6l4NWaLpNzVq",
#         "https://drive.google.com/drive/folders/14ow2HnOtcxF6EcXuxmtlXyMaxkB3Uew1"
# ]
# for id in tqdm(ids2,leave=False):
#   # !/root/.local/bin/gdown {id} --folder -c --output datasets/ --no-cookies &> /dev/null
#   !gdown {id} --folder -c --output datasets/ --no-cookies &> /dev/null

# #census, books ,abt-buy , restaurants
# ids2 = ["https://drive.google.com/drive/folders/1BImatx4-7WrE_eVZ7zPUq408fiUEfq8w",
#         "https://drive.google.com/drive/folders/14ow2HnOtcxF6EcXuxmtlXyMaxkB3Uew1"
#         "https://drive.google.com/drive/folders/1BImatx4-7WrE_eVZ7zPUq408fiUEfq8w",
#         "https://drive.google.com/drive/folders/1wEA-fSQsxLWvu3HlHzPH6l4NWaLpNzVq",
#         "https://drive.google.com/drive/folders/14ow2HnOtcxF6EcXuxmtlXyMaxkB3Uew1",
#         "https://drive.google.com/drive/folders/1yB1_kzYP0HkZzSan7EtzM2MlAvnE_f_t", #dblp acm
#         "https://drive.google.com/uc?id=1JWoXevf_-IU0DbxpCIaO83UK0PRrxd-A"
# ]
# for id in tqdm(ids2,leave=False):
#   # !/root/.local/bin/gdown {id} --folder -c --output datasets/ --no-cookies #| grep 'Download ended unsuccessfully'
#   !gdown {id} --folder -c --output datasets/ --no-cookies #| grep 'Download ended unsuccessfully'

## metodo alternativo

In [ ]:
file_list = [ 
             ('abt-buy','https://storage.live.com/downloadfiles/V1/Zip?authKey=!ANJgdUEuUvK8MX0&application=1141147648','resIds=EA5F5071A63E6E27%21349104&canary=&authkey=%21ANJgdUEuUvK8MX0'),
             ('books','https://storage.live.com/downloadfiles/V1/Zip?authKey=%21ANJgdUEuUvK8MX0&application=1141147648','resIds=EA5F5071A63E6E27%21349105&canary=&authkey=%21ANJgdUEuUvK8MX0'),
            #  ('census','https://storage.live.com/downloadfiles/V1/Zip?authKey=%21ANJgdUEuUvK8MX0&application=1141147648','resIds=EA5F5071A63E6E27%21349106&canary=&authkey=%21ANJgdUEuUvK8MX0'),
             # nao ustilizado# ('dblp-acm','https://storage.live.com/downloadfiles/V1/Zip?authKey=%21ADns5ap%5Fi7hgoVk&application=1141147648','resIds=EA5F5071A63E6E27%21349107&canary=&authkey=%21ADns5ap_i7hgoVk'),
             ('movies','https://storage.live.com/downloadfiles/V1/Zip?authKey=%21ANJgdUEuUvK8MX0&application=1141147648','resIds=EA5F5071A63E6E27%21349108&canary=&authkey=%21ANJgdUEuUvK8MX0'),
            #  ('mvr','https://storage.live.com/downloadfiles/V1/Zip?authKey=%21ANJgdUEuUvK8MX0&application=1141147648','resIds=EA5F5071A63E6E27%21349109&canary=&authkey=%21ANJgdUEuUvK8MX0'),
            #  ('ncvr','',''),
             ('restaurants','https://storage.live.com/downloadfiles/V1/Zip?authKey=%21ANJgdUEuUvK8MX0&application=1141147648','resIds=EA5F5071A63E6E27%21349111&canary=&authkey=%21ANJgdUEuUvK8MX0'),
            #  ('tse','',''),
            #  ('yv-er','',''),
]

for fi in tqdm(file_list):
  context=fi[0]
  url=fi[1]
  param=fi[2]
  !wget -N -O {context}.zip --post-data='{param}' '{url}' 
  !unzip -o {context}.zip -d datasets/{context} 
  

  0%|          | 0/4 [00:00<?, ?it/s]

for details.

--2022-05-28 12:40:16--  https://storage.live.com/downloadfiles/V1/Zip?authKey=!ANJgdUEuUvK8MX0&application=1141147648
Resolving storage.live.com (storage.live.com)... 131.253.14.229
Connecting to storage.live.com (storage.live.com)|131.253.14.229|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘abt-buy.zip’

abt-buy.zip             [                <=> ] 182.91M  13.5MB/s    in 12s     

2022-05-28 12:40:31 (15.8 MB/s) - ‘abt-buy.zip’ saved [191798919]

Archive:  abt-buy.zip
 extracting: datasets/abt-buy/comparison_abt-buy_1000.zip  
 extracting: datasets/abt-buy/comparison_abt-buy_full.zip  
 extracting: datasets/abt-buy/comparison_abt-buy_full_blk_1.zip  
 extracting: datasets/abt-buy/comparison_abt-buy_full_blk_2.zip  
 extracting: datasets/abt-buy/comparison_abt-buy_full_blk_3.zip  
 extracting: datasets/abt-buy/processed_abt-buy.zip  
for details.

--2022-05-28 12:40:33--  https://storage.live.com/do

# Methods

In [ ]:
from sklearn.metrics import mean_absolute_error, matthews_corrcoef

from atuc.utils.file import open_ds, load_data
from atuc.utils import ajustar_treino, showPredictionsSummary , generate_logs_s1
from atuc.utils.misc import testar_limiar
from atuc.stage1 import s1_prepareTrainingData, s1_trainDSClassifier, s1_selectData2Train
from atuc.source_selection import calcular_metricas_dados_relacionado

from atuc.stage1 import coral, apply_source_adptation

## Source and Features Selection Stage

search best source and target distance considering the BenDavid work [1]

In [ ]:
#@title search best source
from atuc.source_selection import search_best_dr

# def search_best_dr(sources,targets,atts = [2,3,4,5,6],lr_model_name='Logistic'):
#     resultado = []


#     for es_ in trange(len(sources),desc='Datasets 1', leave=False):
#         for et_ in trange(len(targets),desc='Datasets 2', leave=False):

#             s_  = sources[es_][0]
#             t_  = targets[et_][0]
#             if s_ != t_:
#                 #print(s_,t_)
#                 _fs = sources[es_][1]
#                 _ft = targets[et_][1]
#                 atts_s = sources[es_][2]
#                 atts_t = targets[et_][2]

#                 for fs in _fs: # itera nos arquivos
#                     for ft in _ft:
#                         if s_ == "census/":
#                             source_,target_,log_ds = load_data(s_,t_,fs,ft,atts_s,atts_t)
#                         elif t_ == "census/":
#                             source_,target_,log_ds = load_data(s_,t_,fs,ft,atts_s,atts_t)
#                         else:
#                             source_,target_,log_ds = load_data(s_,t_,fs,ft,atts_s,atts_t)
                        
#                         try:
#                             et, e_s, dhh,complemento, mcc = calcular_metricas_dados_relacionado(source_,target_,atts=atts,lr_model_name=lr_model_name)    
#                             logs_dr = {'e_s':e_s,'dhh':dhh,'complemento':complemento,'et':et, 'mcc':mcc}
#                         except Exception as e:
# #                         except NameError as e:
#                             nome = e.__class__.__name__
# #                             print(nome)
# #                             print("Error",nome,":",s_,t_,fs,ft)
#                             logs_dr = {'e_s':-1,'dhh':-1,'complemento':-1,'et':-1, 'mcc':-1}
                        
#                         resultado.append({**log_ds , **logs_dr})
#     return resultado

## ATUC

Parametros do metodo

 - *source_*, source dataset
 - *target_*, target datase
 - *ts*, percentual of the datatset that will be employed as test (second filtering)
 - *attributos*, a list with all attributes (similarity metrics) taht will be employed
 - *lr_model_name*, model that will be employed to train the dataset classifier.
   - Logistic: LogisticRegresion
   - SVM: Suport Vector Machine (LinearSVC)
   - DT: Decision Tree (RandomForestClassifier)
 - *_lambda*, the minimal confidence needed to use the pair to train the target (default=.51)
 - *stage1_model*, model that will be employed to train the dataset classifier
   - Logistic: LogisticRegresion
   - SVM: Suport Vector Machine (LinearSVC)
   - DT: Decision Tree (RandomForestClassifier)

In [ ]:
#@title classifier_manufacturing

from atuc.stage1 import execute_classifier_manufacturing
# def execute_classifier_manufacturing(source_,target_,
#                 ts=.3,
#                 atributos=[2,3,4,5,6],columns_y=[7],
#                 _s1_percentualmatch=0.1,
#                 lr_model_name='Logistic',_lambda=.51,
#                 stage1_model='Logistic',
#                 funcao_w=None,
#                 min_match_number=3):

#     assert lr_model_name in ['Logistic','SVM','DT','GBC']
#     assert stage1_model in ['Logistic','SVM','DT','GBC']

#     X_train, X_test, Y_train, Y_test  = s1_prepareTrainingData(source_,target_,ts=.3,atributos=atributos)

#     if lr_model_name == 'Logistic':
#         lr_model = LogisticRegression(random_state=10100,n_jobs=-1)
#     if lr_model_name == 'SVM':
#         svc = LinearSVC(random_state=10100)
#         lr_model = CalibratedClassifierCV(base_estimator=svc, cv=2)
# #         lr_model = SVC(random_state=10100,probability=True)
#     if lr_model_name == 'DT':
#         lr_model = RandomForestClassifier(n_estimators=10,random_state=101010,n_jobs=-1)
#     if lr_model_name == 'GBC':
#         lr_model = GradientBoostingClassifier(random_state=101010)

#     # treinando modelo para dar peso aos pares
#     start_time = time.time()
#     modelo = s1_trainDSClassifier(X_train,Y_train,lr_model)
#     sep_model_time = time.time() - start_time

#     #################################################################
#     ### selecionando os dados para serem utilizados no treinamento
#     ### target
#     #################################################################

#     start_time = time.time()
#     data_ = s1_selectData2Train(modelo,source_,
#                                 lambada_corte=_lambda,
#                                 atributos=atributos)
#     select_train_data_time = time.time() - start_time
    
#     numero_matches = len(data_[data_['is_match'] == 1])  # salvar
#     numero_unmatches = len(data_[data_['is_match'] == 0]) # salvar

#     if numero_matches < min_match_number:
#         raise ValueError("Few Matches " + str(numero_matches))

#     if numero_unmatches < numero_matches * 5:
#         raise ValueError("Few Un Matches " + str(numero_matches) )

#     #### selecionando os dados
#     data_naive = data_.iloc[:,:-1]
#     start_time = time.time()
#     data , w = ajustar_treino(data_,numero_matches,p=_s1_percentualmatch)
#     adjust_train_data_time = time.time() - start_time

#     ###
#     ### regularizacao
#     ###
#     data_adapted = apply_source_adptation(data,target_)
    
    
#     #caso utilizar antes da selecao
#     # data_.iloc[:,:-2]
    
#     # selecao naive
#     Y_naive = data_naive.pop('is_match').values
#     y_s1b = data.pop('is_match')
#     y_s1r = data_adapted.pop('is_match')

#     #treinando classificador no target
#     model_base  = LogisticRegression(random_state=10100,n_jobs=-1)
#     model_s1b   = LogisticRegression(random_state=10100,n_jobs=-1)
#     model_w_s1b = LogisticRegression(random_state=10100,n_jobs=-1)
#     model_s1r   = LogisticRegression(random_state=10100,n_jobs=-1)

#     if stage1_model == 'Logistic':
#         model_base  = LogisticRegression(random_state=10100,n_jobs=-1)
#         model_s1b   = LogisticRegression(random_state=10100,n_jobs=-1)
#         model_w_s1b = LogisticRegression(random_state=10100,n_jobs=-1)
#         model_s1r   = LogisticRegression(random_state=10100,n_jobs=-1)
#     if stage1_model == 'SVM':
#         svc = LinearSVC(random_state=10100)
#         model_base  = CalibratedClassifierCV(base_estimator=svc, cv=2)
#         model_s1b   = CalibratedClassifierCV(base_estimator=svc, cv=2)
#         model_w_s1b = CalibratedClassifierCV(base_estimator=svc, cv=2)
#         model_s1r   = CalibratedClassifierCV(base_estimator=svc, cv=2)
#     if stage1_model == 'DT':
#         model_base  = RandomForestClassifier(n_estimators=10,random_state=101010,n_jobs=-1)
#         model_s1b   = RandomForestClassifier(n_estimators=10,random_state=101010,n_jobs=-1)
#         model_w_s1b = RandomForestClassifier(n_estimators=10,random_state=101010,n_jobs=-1)
#         model_s1r   = RandomForestClassifier(n_estimators=10,random_state=101010,n_jobs=-1)
#     if lr_model_name == 'GBC':
#         model_base  = GradientBoostingClassifier(random_state=101010,loss="exponential")
#         model_s1b   = GradientBoostingClassifier(random_state=101010,loss="exponential")
#         model_w_s1b = GradientBoostingClassifier(random_state=101010,loss="exponential")
#         model_s1r   = GradientBoostingClassifier(random_state=101010,loss="exponential")
    

#     model_base.fit(data_naive,Y_naive.astype('int'))
    
#     start_time = time.time()
#     model_s1b.fit(data,y_s1b.astype('int'))
#     train_classifier_time = time.time() - start_time

#     model_s1r.fit(data_adapted,y_s1r.astype('int')) # ajustado
    

#     ######################################
#     ### Classificando
#     ######################################
#     alvo = atributos + columns_y

#     t_ = target_.iloc[:,alvo]
#     Y_t = t_.pop('is_match').values

#     Y_naive = modelo.predict(t_) # modelo totalmente treinado no source
#     Y_base = model_base.predict(t_)
    
#     start_time = time.time()
#     Y_topk = model_s1b.predict(t_)
#     execute_classifier_time = time.time() - start_time
    
#     Y_topk_adpt = model_s1r.predict(t_)

#     naive_log = generate_logs_s1(Y_t, Y_naive,'naive','all',lr_model_name,stage1_model)
#     base_log = generate_logs_s1(Y_t, Y_base,'tl-base','all',lr_model_name,stage1_model)
#     topk_log = generate_logs_s1(Y_t, Y_topk,'tl_pprl','top_k',lr_model_name,stage1_model)
#     topk_adpt_log = generate_logs_s1(Y_t, Y_topk_adpt,'tl_pprl','top_k_adpt',lr_model_name,stage1_model)

#     logs = [naive_log,base_log,topk_log,topk_adpt_log]

#     # passando funcao com peso para o modelo
#     # TODO: Colocar no log a funcao
#     if funcao_w == None:
#         model_w_s1b.fit(data,y_s1b.astype('int'),sample_weight=w)
#         Y_topkw = model_w_s1b.predict(t_)
#         topkw_log = generate_logs_s1(Y_t, Y_topkw,'tl_pprl-w','top_k',lr_model_name,stage1_model)
#     else:
#         for fx in funcao_w:
#             w_linha = fx(w)
#             model_w_s1b.fit(data,y_s1b.astype('int'),sample_weight=w_linha)

#             try:
#                 fw = fx.__name__
#             except AttributeError:
#                 fw = 'naosei'

#             Y_topkw = model_w_s1b.predict(t_)
# #             print("\t" + lr_model_name)
#             topkw_log = generate_logs_s1(Y_t, Y_topkw,'tl_pprl-w','top_k',lr_model_name,stage1_model,peso=fw)
#             logs.append(topkw_log)

#     # nesses dados estao faltando o adptados
#     _dnaive = target_.copy()
#     _dbase = target_.copy()
#     _dtopk = target_.copy()
#     _dtopkw = target_.copy() # pega so o ultimo kw

#     _dnaive['y'] = Y_naive
#     _dbase['y']  = Y_base
#     _dtopk['y']  = Y_topk
#     _dtopkw['y'] = Y_topkw

#     data = [_dnaive,_dbase,_dtopk,_dtopkw]
    
#     time_log = {'sep_model_training':sep_model_time,
# #                 'exec_sep_model' : execute_classifier_time,
#                 'sep_model_execution':select_train_data_time,
#                 'instance_adptation':adjust_train_data_time,
#                 'train_classifier':train_classifier_time,
#                 'execute_classifier':execute_classifier_time}
    
# #     sep_model_time, select_train_data_time, adjust_train_data_time, train_classifier_time, execute_classifier_time

#     return data, logs, time_log#, data_ # colocado o data

## Unsupervised Domain Adaptation

In [ ]:
def executar_s1(sources,targets,alg,
                 lr_model_name=['Logistic'],
                 metricas_de_similaridade=[2,3,4,5,6],
                 percetual_teste=.1,
                 pesos=None,s1_percentualmatch=.01,
                best_threshold=True):
    resultado = []
    resultado_time = []

    for es_ in trange(len(sources),desc='Source Datasets', leave=False):
        for et_ in trange(len(targets),desc='Target Datasets', leave=False):

            s_  = sources[es_][0]
            t_  = targets[et_][0]
            if s_ != t_:
                #print(s_,t_)
                _fs = sources[es_][1]
                _ft = targets[et_][1]
                atts_s = sources[es_][2]
                atts_t = targets[et_][2]
                blk_s = sources[es_][3]
                blk_t = targets[et_][3]

#                 print(s_,t_,_fs,_ft,atts_s,atts_t)

                for fs in _fs: # itera nos arquivos
                    for ft in _ft:
                        source_,target_,log_ds = load_data(s_,t_,fs,ft,atts_s,atts_t,
                                                            blk_s=blk_s,blk_t=blk_t,
                                                            ds_dir='/content/datasets')

                        # if best_threshold:
                        #     logs_th = testar_limiar(target_,_metricas=metricas_de_similaridade)

                        for a2 in tqdm(alg,desc='Algo loop', leave=False):
                            for td in lr_model_name:
#                                 print(td)
                                try:
                                    data_array,logs_s1,time_log = execute_classifier_manufacturing(source_,target_,
                                                                     atributos=metricas_de_similaridade,columns_y=[7],
                                                                     _s1_percentualmatch=s1_percentualmatch,
                                                                       ts=percetual_teste,
                                                                       lr_model_name=td, stage1_model=a2,
                                                                       funcao_w=pesos)

                                    time_temp = {'sourcet':s_,'targett':t_,'features':str(metricas_de_similaridade),
                                                 'nfeatures':len(metricas_de_similaridade),
                                                 'separationmodel':td, 'classifier': a2
                                                }
                                    time_data = dict(time_temp, **time_log)
                                    resultado_time.append(time_data)

                                    del data_array
                                    #armazena os logs
                                    for l in logs_s1:
                                        # resultado.append({**log_ds , **l})
                                        #TESTAR ESSE AQUI
                                        resultado.append({**log_ds ,**time_temp ,**l})
                                except Exception as e:
                                # except NameError as e:
                                    nome = e.__class__.__name__
                                    logging.debug("Error :: ",nome," :: ",td," - ",s_,t_,fs,ft)
                                    logging.debug(e)

#                             except TypeError as e:
    #                             print("Error",dss[s_],dss[t_],files[s_],files[t_],td,a2)
                        
                        if best_threshold:
                            logs_th = testar_limiar(target_,_metricas=metricas_de_similaridade)
                            time_temp = {'sourcet':s_,'targett':t_,
                                         'features':str(metricas_de_similaridade),
                                          'nfeatures':len(metricas_de_similaridade),
                                          'separationmodel':'none',
                                          'classifier': 'threshold'
                                          }
                            for log_th in logs_th:
                                resultado.append({**log_ds , **time_temp, **log_th})
                        else:
                            resultado.append({**log_ds})
                        
    return resultado, resultado_time

## salvar resultados

In [ ]:
def write_results_2_google(df, google_auth,
                  spreadsheetId='invalid_id',
                  sheetName = 'Sheet1'):
  
  """
  auth.authenticate_user()
  creds, _ = default()
  gc = gspread.authorize(creds) #google_auth
  """

  sh = google_auth.open_by_key(spreadsheetId)
  sheet = sh.worksheet(sheetName)

  values = df.values.tolist()
  #append_row
  sh.values_append(sheetName, {'valueInputOption': 'USER_ENTERED'}, {'values': values})

# Execute experiment

## setup

### learning parameters

In [ ]:
# features_space = [ 
#         [2,3],#[3,4],[3,5],[3,6],[4,6],[4,5],[5,6],
#         [3,4,5],#[4,5,6],[3,5,6],[2,3,4],[2,3,6],
#         [2,3,4,5],#[2,3,4,6],[3,4,5,6],[2,4,5,6],
#         [2,3,4,5,6]
#     ]

features_space = [ 
        [2,3,4,5,6],
        [2,3,4,5],[2,3,4,6],[3,4,5,6],[2,4,5,6],
        [2,3,4],[2,3,5],[2,3,6],[3,4,5],[3,5,6],[4,5,6]
        # [2,3],[3,4],[3,5],[3,6],[4,6],[4,5],[5,6]
]

alg = ['GBC','Logistic','SVM','DT']
alg = ['GBC','Logistic','SVM']

# alg_dt = ['GBC','Logistic','SVM','DT']
# alg = ['Logistic']
# alg_dt = ['Logistic','SVM']
alg_dt = ['SVM'] #separation model
pm = .005 # oque é isso mesmo?

### Dataset Utilizados

In [ ]:
###
### Source
###

ds_source_all_blk = [
            ("abt-buy",['comparison_abt-buy_full_blk_1.zip'],3,None),
            # ("dblp-acm",['comparison_dblp-acm_full_blk_1.zip'],4,None),
            ("books",['comparison_books_amazon-barnesnobel_blk_1.zip','comparison_books_amazon-barnesnobel-small_blk_1.zip'],3,None),
            ("books",['comparison_books_goodreads-barnesnobel_blk_1.zip'],4,None),
#             ("books/",['comparison_books_goodreads-barnesnobel.zip'],5),
            ("movies",['comparison_movies_imdb-rottentomatos_blk_1.zip'],3,None),
            ("movies",['comparison_movies_imdb-tmd_blk_1.zip'],2,None),
            ("restaurants",['comparison_restaurants_fodors-zagats_blk_1.zip'],4,None),
            # ("restaurants",['comparison_restaurants_yelp-yellowpages_blk_1.zip'],5,None),
#             ("restaurants/",['comparison_restaurants_yelp-yellowpages.zip'],6),
            # ("restaurants",['comparison_restaurants_yelp-zomato_blk_1.zip'],3,None),
          ]
###
### Target 
###

#context, file list, atts , blk
ds_target_blk = [                 
            ("ncvr",['comparison_ncvoters_10000_0.1_blk.zip'],3,2),
            ("mvr",['comparison_michiganvoters_10000_0.1_blk.zip'],3,2),
            # ("tse",['comparison_processed_tse-10000-8_blk.zip'],3,3), #blk?4 $ PROBLEMA
            ("tse",['comparison_tse_2000_0.1_blk_1.zip'],3,None), #blk?4 $ PROBLEMA
            ("census",['comparison_census_blk.zip'],4,2), #blk-2
            ("yv-er",['comparison_processed_yver_blk.zip'],4,2),#blk-2
            ("yv-er",['comparison_yver_full.zip'],4,None)#blk-2
            
          ]

### definir scenario

In [ ]:
scenario = 'blk_blk'
# scenario = '2'
ds_source = ds_target_blk + ds_source_all_blk

ds_target = ds_target_blk
# ds_target = ds_target_full

#autentica no google
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

## **run**

### run : Unsupervised classification

In [ ]:
saida_tempo = []
saida_qualidade = []

#todo: resolver problema do limiar
for features in tqdm(features_space,desc='features',leave=False):
    r,rt = executar_s1(ds_source,ds_target,alg,lr_model_name=alg_dt,percetual_teste=.1,
                     s1_percentualmatch = pm,best_threshold=True,
                     metricas_de_similaridade=features)
    
    dft = pd.DataFrame(rt)
    dfq = pd.DataFrame(r)

    if len(dfq.columns) > 15:
      a = dfq.fillna(-1)
      write_results_2_google(a,gc,
                    spreadsheetId='1AhdK5O6oZFHZSXLsBPI59YDj9GSF8AGJzNTnCi12hqM',
                    sheetName='qualidade_'+scenario)
      write_results_2_google(dft,gc,
                    spreadsheetId='1AhdK5O6oZFHZSXLsBPI59YDj9GSF8AGJzNTnCi12hqM',
                    sheetName='tempo_'+scenario)

### run : Source Selection

In [ ]:
saida_ss = []

# ds_target = ds_target_blk
# ds_source = ds_target_blk

# ds_target = ds_target_full
# ds_source = ds_target_full + ds_source_all_full

# ds_source =  [ ds_target_blk[2] ] # - census / 2 - mvr
# ds_target =  [ ds_target_blk[0] ]

for features in tqdm(features_space,desc='features',leave=False):
    rf = search_best_dr(ds_source,ds_target,atts=features)
    df = pd.DataFrame(rf)
    # adiciona os dados dos features considerados
    df['features'] = str(features)
    df['n_features'] = len(features)
    saida_ss.append(df)

    if len(df.columns) > 4:
      write_results_2_google(df,gc,
                    spreadsheetId='1AhdK5O6oZFHZSXLsBPI59YDj9GSF8AGJzNTnCi12hqM',
                    sheetName='dhcc_'+scenario)

features:   0%|          | 0/11 [00:00<?, ?it/s]

Datasets 1:   0%|          | 0/12 [00:00<?, ?it/s]

Datasets 2:   0%|          | 0/6 [00:00<?, ?it/s]

Datasets 2:   0%|          | 0/6 [00:00<?, ?it/s]

Datasets 2:   0%|          | 0/6 [00:00<?, ?it/s]

...

Datasets 2:   0%|          | 0/6 [00:00<?, ?it/s]

Datasets 2:   0%|          | 0/6 [00:00<?, ?it/s]

Datasets 2:   0%|          | 0/6 [00:00<?, ?it/s]

Datasets 2:   0%|          | 0/6 [00:00<?, ?it/s]

Datasets 2:   0%|          | 0/6 [00:00<?, ?it/s]

# Reference

1. Ben-David, John Blitzer, Koby Crammer, Alex Kulesza, Fernando Pereira, and Jennifer Wortman Vaughan. A theory of learning from different domains. Machine Learning, 79(1-2):151–175, 5 2010. ISSN 0885- 6125. doi: 10.1007/s10994-009-5152-4.